## Import librerie

Before training the model all the required modules have to be installed

In [ ]:
!pip install -r .//yolov7//requirements.txt

To install pytorch with CUDA support run the following cell:

In [ ]:
!pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116

In [1]:
import requests, zipfile, io, shutil, os, torch, pycocotools, yaml
import pandas as pd
import ipywidgets as widgets
import numpy as np
import cv2

## Clone Yolo v7 repo

The repository being cloned is a fork of the [Yolov7](https://github.com/WongKinYiu/yolov7) repository with customized settings.
The following settings have been customized for the purpose of this project:
- `cfg/training/yolov7.yaml` update the classes number to 52
- `data/coco.yaml`: changed the path of the datasets and updated the number and value of the classes
- `data/data.yaml`: it's a copy of the coco.yaml file; such copy was created because using the original file would trigger some tasks specific for the coco dataset
- `requirements.txt`: replace the default torch version with the CUDA enabled one

In [ ]:
!git clone https://github.com/Filocava99/Yolov7-Poker-Hands-classification.git yolov7

Downloading the compiled Yolo v7 model and its less complex version (not used in the final/production version)

In [ ]:
yolo_v7 = "https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt"
yolo_v7_tiny = "https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-tiny.pt"
r = requests.get(yolo_v7)
with open('./yolov7/yolov7.pt', "wb") as f:
    f.write(r.content)
r = requests.get(yolo_v7_tiny)
with open('./yolov7/yolov7_tiny.pt', "wb") as f:
    f.write(r.content)

## Import del dataset
Questo step è necessario da eseguire anche in caso si voglia saltare la fase di training ed eseguire solamente quella di test.

In [2]:
obj_detection_dataset_url = "https://doc-0s-5o-docs.googleusercontent.com/docs/securesc/sg8gvb6dhfns7p2hsog47t7fftck065l/21hr77ig1fhrjs46tleuq8sth2mkjttv/1682870325000/05406259354737088254/05406259354737088254/1GOJ9REocmncYz5Fc_bXgF3vb313EJrsh?e=download&ax=ADWCPKDx0E5l-qDhkmKmj9gi5cl2obzYHiMHULwirSpZValYCYcbntrSZsIRz-96-Yl5WCej4fLbISKPkfKJrlQimfnFqesQdzJvcLPLWumJU8PtNSJHniQKE1NGT0yt8sL2y-3TBiFeFiXSioUBoSWF28OYQdgxxkUNWXl8LRMXTpJQdMbbf0qQzf6gvyqsSPZTAeOPvm2jdtGlgiQ2M6gM9EC53n2YUoI2K9_1nwynz_T-UK49u7ZhU3C43UNz_KScPljnLUPH2IfqjtAI6lf2Z1nDrQ_kAQ8KOLT2pOGkQl2_9PRyfdIsalXbRDs10emHUzdhIzNS1sYHhm7JzB9ut3rdoBQgFEe4Jv3eSZS2ZxsYhcihvv37xgcMDq-mDnnP9oY-INZGahscSTy5vC_Iw2lmvA8g-kDC00qoB5ki4EXLjvvUivh4xV7z2mh0l2EqojCvqCNafprFoNXuRN4RExsoPFxWJEQBnEYgV-mdMIglHA7qadHN08yU-h__qDj3639slKQUpvSwyCtAn5Wnrtp2kwSLG80v8iO5C_pBh-F8-aBMZApfyfFauv_HuEsl0JjGVN5ARyzupYVdwJk9k-XKS-YAHWQLO2wgv55r0A3M41-fm0wM2OyoGTViJPjJho6ET8oR6jjZdMSmYtwMPp94XSlwEZQ6yCR_XMkMODSRcoNC5vTJukBws6yp6NGDdIFZJ5nEeb7rU4fMXzxbX8inPIDZhV1izd4I-C-l4fDM-sgau1QGCPOdSj87C7dCfnBel4588E6PnQvkeOfQgHZ80tPxhGVYRBx9G_j-KhNqD1ujZQv8Qv8wOjfbUZbKZM47tnM4YYdLErP6MApsPKlMGCJpQy0F2FldGfBimJd99OzU2vhKjAd5N9WC4vjlEeR6KRVu_9KCa2dRHGgwmaL1Q8fUddQQJshFLGWrVI0sogE5EUsRI3eJN5Yoo_URjQN_46TnFw&uuid=7a4f4766-2708-4cde-9121-0d127e215772&authuser=0"
#"https://www.kaggle.com/datasets/andy8744/playing-cards-object-detection-dataset/download?datasetVersionNumber=4"

if(os.path.exists("./yolov7/dataset") == False):
    r = requests.get(obj_detection_dataset_url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall("./yolov7/dataset")
    
from yaml.loader import SafeLoader
classes = []
# Open the file and load the file
with open('./yolov7/dataset/data.yaml') as f:
    classes = yaml.load(f, Loader=SafeLoader)["names"]
print("Total classes: " + str(len(classes)))
print("Classes: " + str(classes))

Total classes: 52
Classes: ['10c', '10d', '10h', '10s', '2c', '2d', '2h', '2s', '3c', '3d', '3h', '3s', '4c', '4d', '4h', '4s', '5c', '5d', '5h', '5s', '6c', '6d', '6h', '6s', '7c', '7d', '7h', '7s', '8c', '8d', '8h', '8s', '9c', '9d', '9h', '9s', 'Ac', 'Ad', 'Ah', 'As', 'Jc', 'Jd', 'Jh', 'Js', 'Kc', 'Kd', 'Kh', 'Ks', 'Qc', 'Qd', 'Qh', 'Qs']


> Gli step successivi sono da considerarsi specifici per la fase di training. Essendo il modello già addestrato è possibile saltare i passaggi da quì fino al 

## Remove roboflow junk from file names

In [19]:
import os 

# remove roboflow extra junk

count = 0
for i in sorted(os.listdir('./yolov7/dataset/train/labels')):
    if count >=3:
        count = 0
    count += 1
    if i[0] == '.':
        continue
    j = i.split('_')
    dict1 = {1:'a', 2:'b', 3:'c'}
    source = './yolov7/dataset/train/labels/'+i
    dest = './yolov7/dataset/train/labels/'+j[0]+dict1[count]+'.txt'
    os.rename(source, dest)
    
count = 0
for i in sorted(os.listdir('./yolov7/dataset/train/images')):
    if count >=3:
        count = 0
    count += 1
    if i[0] == '.':
        continue
    j = i.split('_')
    dict1 = {1:'a', 2:'b', 3:'c'}
    source = './yolov7/dataset/train/images/'+i
    dest = './yolov7/dataset/train/images/'+j[0]+dict1[count]+'.jpg'
    os.rename(source, dest)
    
for i in sorted(os.listdir('./yolov7/dataset/valid/labels')):
    if i[0] == '.':
        continue
    j = i.split('_')
    source = './yolov7/dataset/valid/labels/'+i
    dest = './yolov7/dataset/valid/labels/'+j[0]+'.txt'
    os.rename(source, dest)
    
for i in sorted(os.listdir('./yolov7/dataset/valid/images')):
    if i[0] == '.':
        continue
    j = i.split('_')
    source = './yolov7/dataset/valid/images/'+i
    dest = './yolov7/dataset/valid/images/'+j[0]+'.jpg'
    os.rename(source, dest)
for i in sorted(os.listdir('./yolov7/dataset/test/labels')):
    if i[0] == '.':
        continue
    j = i.split('_')
    source = './yolov7/dataset/test/labels/'+i
    dest = './yolov7/dataset/test/labels/'+j[0]+'.txt'
    os.rename(source, dest)
    
for i in sorted(os.listdir('./yolov7/dataset/test/images')):
    if i[0] == '.':
        continue
    j = i.split('_')
    source = './yolov7/dataset/test/images/'+i
    dest = './yolov7/dataset/test/images/'+j[0]+'.jpg'
    os.rename(source, dest)

## Train the dataset

To train the dataset we use the following parameters:
- workers: 8, adjust to the number of CPU cores
- device: 0, where 0 is the id of the GPU
- batch size: 32
- epochs: 100
- img: 416 416, the size of the images of the dataset
- hyp: data/hyp.scratch.custom.yaml, the file containing the hyperparameters to use
- name: yolov7-custom, the name to use to save the trained model
- weights: yolov7.pt, the weights to start from

In [ ]:
!python .//yolov7//train.py --workers 8 --device 0 --batch-size 32 --epochs 100 --img 416 416 --hyp data//hyp.scratch.custom.yaml --name yolov7-custom --weights yolov7.pt

After the training we can run the model to detect the cards on different sources. In the first case we are providing a video while in the second case we are using the webcam feed stream.

In [ ]:
!python .//yolov7//detect.py --weights .//yolov7//runs//train//yolov7-custom//weights//best.pt --conf 0.5 --img-size 1280 --source .//dataset//test//videotest.mp4 --view-img --no-trace 

In [ ]:
!python .//yolov7//detect.py --weights .//yolov7//runs//train//yolov7-custom//weights//best.pt --conf 0.5 --img-size 1280 --source 0 --view-img --no-trace 

## Test evaluation

In [100]:
# Load the image
image_path = './yolov7/runs/train/yolov7-custom/results.png'
with open(image_path, 'rb') as f:
    image = f.read()

# Create the image widget
image_widget = widgets.Image(
    value=image, 
    format='png',
    width=1920
)
display(image_widget)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\t`\x00\x00\x04\xb0\x08\x06\x00\x00\x00\x7f\xa4q\x81\…

I grafici mostrati nella cella precedente sono stati generati direttamente dall'implementazione di Yolo 7 che è stata utilizzata. Sono di particolare interesse i grafici di precision, recall e mAP, sia con IoU threshold 0.5 che 0.95.
The graphs shown in the previous cell were generated directly from the implementation of Yolo 7 that was used. Of particular interest are the precision, recall, and mAP graphs, both with an IoU threshold of 0.5 and 0.95.

The model converged around the 30th epoch, but we continued training up to 100 epochs to verify any potential improvements. The mAP improved slightly, but there were no issues of overfitting, so overall the subsequent epochs were not considered useless.

## Score evaluation algorithm
This algorithm will return a tuple of two elements, the first element is an integer between 1 and 10 representing the point value of the hand and the second element is the highest card in case of tie-breaker.
The point values are:
- 10 for Straight Flush
- 8 for Four of a Kind
- 7 for Full House
- 6 for Flush
- 5 for Straight
- 4 for Three of a Kind
- 3 for Two Pairs
- 2 for One Pair
- 1 for High Card

The expected input for this algorithm is a list of five strings, where each string represents a poker card in the format of "value suit". The value can be one of the following: '2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A'. The suit can be one of the following: 'C' for Clubs, 'D' for Diamonds, 'H' for Hearts, 'S' for Spades.
```python
cards = ["2 H", "3 H", "4 H", "5 H", "6 H"]
poker_hand(cards)
```

In [101]:
import collections

points = {
    10: "Straight flush",
    8: "Four of a kind",
    7: "Full house",
    6: "Flush",
    5: "Straight",
    4: "Three of a kind",
    3: "Two pairs",
    2: "One pair",
    1: "High card"
}

def poker_hand(cards):
    # Create a dictionary to store the count of each card value
    card_count = collections.defaultdict(int)
    for card in cards:
        value = card[0]
        if value == 'T':
            value = '10'
        elif value == 'J':
            value = '11'
        elif value == 'Q':
            value = '12'
        elif value == 'K':
            value = '13'
        elif value == 'A':
            value = '14'
        card_count[value] += 1
    # Check for a flush (all cards are the same suit)
    flush = all(card[2] == cards[0][2] for card in cards)
    # Check for a straight (cards are in sequential order)
    straight = (max([int(x) for x in card_count.keys()]) - min([int(x) for x in card_count.keys()]) == 4) and (len(card_count) == 5)
    # Check for a straight flush (flush and straight)
    straight_flush = flush and straight
    # Check for four of a kind
    four_of_a_kind = any(count == 4 for count in card_count.values())
    # Check for three of a kind
    three_of_a_kind = any(count == 3 for count in card_count.values())
    # Check for a full house (three of a kind and one pair)
    full_house = three_of_a_kind and any(count == 2 for count in card_count.values())
    # Check for two pairs
    two_pairs = len([count for count in card_count.values() if count == 2]) == 2
    # Check for one pair
    one_pair = len([count for count in card_count.values() if count == 2]) == 1
    if straight_flush:
        return (points[10], max(card_count.keys()))
    elif four_of_a_kind:
        return (points[8], max(card_count, key=lambda x: card_count[x]))
    elif full_house:
        return (points[7], max(card_count, key=lambda x: card_count[x]))
    elif flush:
        return (points[6], max(card_count.keys()))
    elif straight:
        return (points[5], max(card_count.keys()))
    elif three_of_a_kind:
        return (points[4], max(card_count, key=lambda x: card_count[x]))
    elif two_pairs:
        return (points[3], max(card_count, key=lambda x: card_count[x]))
    elif one_pair:
        return (points[2], max(card_count, key=lambda x: card_count[x]))
    else:
        return (points[1], max(card_count.keys()))

You can select a photo of a poker hand to test out the model:

In [34]:
uploader = widgets.FileUpload(multiple=False)
display(widgets.HBox([uploader]))

In [117]:
fileName = list(uploader.value[0].items())[0][1]
content = list(uploader.value[0].items())[3][1]
widgets.Image(value=content)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xe2\x02(ICC_PROFILE\x00\x01\x…

In [116]:
decoded = cv2.imdecode(np.frombuffer(content, np.uint8), -1)
width = decoded.shape[1]
height = decoded.shape[0]
max_size = max(width, height)
print("Width: " + str(width))
print("Height: " + str(height))
print("If you want to use the webcam feed instead of the uploaded image please check the following checkbox:")
checkbox = widgets.Checkbox(
    value=False,
    description='Use webcam',
    disabled=False
)
display(checkbox)

Width: 720
Height: 1280
If you want to use the webcam feed instead of the uploaded image please check the following checkbox:


Checkbox(value=False, description='Use webcam')

In [ ]:
if(os.path.exists("./runs/detect/custom")):
    shutil.rmtree("./runs/detect/custom")
os.makedirs("./runs/detect/custom")
with open("./runs/detect/custom/test.jpg", "wb") as f:
    f.write(content)
if(checkbox.value):
    !python .//yolov7//detect.py --weights .//yolov7//runs//train//yolov7-custom//weights//best.pt --conf 0.5 --img-size 1280 --source 0 --view-img --no-trace 
else:
    !python .//yolov7//detect.py --weights .//yolov7//runs//train//yolov7-custom//weights//best.pt --save-txt --conf 0.5 --img-size {max_size} --source .//runs//detect//custom//test.jpg --name custom --exist-ok --view-img

In [80]:
df = pd.read_csv("./runs/detect/custom/labels/test.txt", delim_whitespace=True, header=None)
objs = []
for index, row in df.iterrows():
    name = str(classes[int(row[0])])
    name = name[0] + " " + str.upper(name[1])
    if((name in objs) == False):
        objs.append(name)
print(objs)

['8 D', '7 H', '5 D', '9 C', '6 D']


In [119]:
hand = poker_hand(objs)
print("Point: " + hand[0])
print("Highest card value: " + str(hand[1]))

Point: Straight
Highest card value: 9


## Final thoughs

The Yolo v7 model outperfomed the SSD model reaching a mAP of over 95%, while SSD only topped 5%. There are still a few issues, probably related to the dataset used; in particular, the model struggles to detect the values of the cards when the photo has been taken to close to them (some background has to be visible for the model to detect cards). Another issue, which is instead caused by the scoring algorithm, is that the same card may be counted twice when all of its corners are uncovered; as a design implementation I decided not to handle it, to better showcase how the model predictions work.